假定了特征与特征之间相互独⽴的⻉叶斯公式。<br/>
P(C)：每个⽂档类别的概率(某⽂档类别数／总⽂档数量)<br/>
 P(W│C)：给定类别下特征（被预测⽂档中出现的词）的概率<br/>
计算⽅法：P(F1│C)=Ni/N	（训练⽂档中去计算）<br/>
Ni为该F1词在C类别所有⽂档中出现的次数<br/>
N为所属类别C下的⽂档所有词出现的次数和<br/>
P(F1,F2,…)	预测⽂档中每个词的概率<br/>
如果计算两个类别概率⽐较：<br/>
所以我们只要⽐较前⾯的⼤⼩就可以，得出谁的概率⼤<br/>

拉普拉斯平滑系数<br/>
系数一般指定为1，m(分母)为训练文档中统计出的特征词个数

.朴素⻉叶斯优缺点<br/>
优点：<br/>
朴素⻉叶斯模型发源于古典数学理论，有稳定的分类效率<br/>
对缺失数据不太敏感，算法也⽐较简单，常⽤于⽂本分类<br/>
分类准确度⾼，速度快<br/>
缺点：<br/>
由于使⽤了样本属性独⽴性的假设，所以如果特征属性有关联时其效果不好<br/>
需要计算先验概率，⽽先验概率很多时候取决于假设，假设的模型可以有很多种，因此在某些时候会由于假设<br/>
的先验模型的原因导致预测效果不佳<br/>

朴素⻉叶斯是⽣成模型，<br/>
根据已有样本进⾏⻉叶斯估计学习出先验概率P(Y)和条件概率P(X|Y)，
进⽽求出联合分布概率P(XY),
最后利⽤⻉叶斯定理求解P(Y|X)，<br/>
⽽LR是判别模型，
根据极⼤化对数似然函数直接求出条件概率P(Y|X)；<br/>


朴素⻉叶斯是基于很强的条件独⽴假设（在已知分类Y的条件下，各个特征变量取值是相互独⽴的），<br/>
⽽LR则对此没有要求；<br/>

朴素⻉叶斯适⽤于数据集少的情景，<br/>
⽽LR适⽤于⼤规模数据集。<br/>

商品评论情感分析

In [2]:
import pandas as pd
import numpy as np
import jieba
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
#1）获取数据
data=pd.read_csv('data/书籍评价.csv',encoding='gbk')
data

,Unnamed: 0,内容,评价
0,0,从编程小白的角度看，入门极佳。,好评
1,1,很好的入门书，简洁全面，适合小白。,好评
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评
4,4,看了一遍还是不会写，有个概念而已,差评
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评
7,7,破书一本,差评
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评
9,9,基础知识写的挺好的！,好评


In [8]:
#2）数据基本处理
#2.1）	取出内容列，对数据进⾏分析
content=data['内容']
#2.2）	判定评判标准
data.loc[data.loc[:,'评价']=='好评','评论编号']=1
data.loc[data.loc[:,'评价']=='差评','评论编号']=0
#2.3）	选择停⽤词
stopWords=[]
with open('data/stopwords.txt','r',encoding='utf-8') as f:  
    for line in f:
        stopword=line.strip()
        stopWords.append(stopword)
print(stopWords)        

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', '..', '...', '......', '...................', './', '.一', '.数', '.日', '/', '//', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '://', '::', ';', '<', '=', '>', '>>', '?', '@', 'A', 'Lex', '[', '\\', ']', '^', '_', '`', 'exp', 'sub', 'sup', '|', '}', '~', '~~~~', '·', '×', '×××', 'Δ', 'Ψ', 'γ', 'μ', 'φ', 'φ．', 'В', '—', '——', '———', '‘', '’', '’‘', '“', '”', '”，', '…', '……', '…………………………………………………③', '′∈', '′｜', '℃', 'Ⅲ', '↑', '→', '∈［', '∪φ∈', '≈', '①', '②', '②ｃ', '③', '③］', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '──', '■', '▲', '', '、', '。', '〈', '〉', '《', '》', '》），', '」', '『', '』', '【', '】', '〔', '〕', '〕〔', '㈧', '一', '一.', '一一', '一下', '一个', '一些', '一何', '一切', '一则', '一则通过', '一天', '一定', '一方面', '一旦', '一时', '一来', '一样', '一次', '一片', '一番', '一直', '一致', '一般', '一起', '一转眼', '一边', '一面', '七', '万一', '三', '三天两头', '三番两次', '三番五次', '上', '上下', '上升', '上去', '上来', '上述', '上面', '下', '下列', '下去', '下来', '下面', '不', '不一', '不下', '不久', 

In [11]:
#2.4）	把内容处理，转化成标准格式
com_list=[]
for i in content:
    seg=jieba.cut(i,cut_all=False)
    seg=','.join(seg)
    com_list.append(seg)
#2.5）	统计词的个数
#CountVectorizer	类会将⽂本中的词语转换为词频矩阵
con=CountVectorizer(stop_words=stopWords)
x=con.fit_transform(com_list)
name=con.get_feature_names_out()
print(x.toarray())	
print(name)

[[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
  0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
  0]
 [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1
  1]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0
  0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 

D:\wym\tool\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


In [12]:
#2.6）准备训练集和测试集,数据集太少，前10行训练，后三行厕所
x_train=x.toarray()[:10,:]
y_train=data['评价'][:10]
x_test=x.toarray()[10:,:]
y_test=data['评价'][10:]
#3）模型训练
mb=MultinomialNB(alpha=1) # 平滑系数
mb.fit(x_train,y_train)
y_predict=mb.predict(x_test)
print(y_predict)
print(y_test)

['差评' '差评' '差评']
10    差评
11    差评
12    差评
Name: 评价, dtype: object


In [13]:
#4）模型评估
mb.score(x_test,y_test)

1.0